# Week 6 Problem Set

## Homeworks

In [29]:
%load_ext nb_mypy
%nb_mypy On

The nb_mypy extension is already loaded. To reload it, use:
  %reload_ext nb_mypy


In [30]:
from typing import TypeAlias
from typing import Optional, Any, Callable, Iterator, Iterable, Reversible, cast
from __future__ import annotations

Number: TypeAlias = int | float

**HW1.** Write a function `get_neighbours(graph, vert)` which returns a list of all neighbours of the requested Vertex `vert` in the `graph`. Return `None` if the Vertex is not in the graph.

In [31]:
def get_neighbours(graph: dict[str, list[str]], vert: str) -> Optional[list[str]]:
    # Attempt to retrieve the list of neighbours for the given vertex 'vert' from the graph
    # If the vertex is not found in the graph, return None
    return graph.get(vert, None)

In [32]:
graph: dict[str, list[str]] = {"A": ["B", "C"], 
         "B": ["C", "D"],
         "C": ["D"],
         "D": ["C"], 
         "E": ["F"],
         "F": ["C"]}

In [33]:
assert get_neighbours(graph, "B") == ["C", "D"]
assert get_neighbours(graph, "A") == ["B", "C"]
assert get_neighbours(graph, "F") == ["C"]
assert get_neighbours(graph, "Z") == None
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [34]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###
###
### AUTOGRADER TEST - DO NOT REMOVE
###


**HW2.** Write a function called `get_path()` which takes in the tree generated by BFS algorithm containing the parent vertices information, start vertex and end vertex. The function should output a list of string containing the path from the starting vertex to the ending vertex.

In [35]:
def get_path(start: str, end: str, parent_tree: dict[str, Optional[str]]) -> list[str]:
    path: list[str] = []  # Initialize an empty list to store the path
    current: Optional[str] = end  # Start from the end vertex
    while current is not None:  # Traverse the parent tree until reaching the start vertex or None
        path.append(current)  # Add the current vertex to the path
        current = parent_tree.get(current)  # Move to the parent of the current vertex
    path.reverse()  # Reverse the path to get it from start to end
    if path[0] == start:  # Check if the path starts with the start vertex
        return path  # Return the valid path
    else:
        return []  # Return an empty list if no valid path is found

In [36]:
parent_tree: dict[str, Optional[str]] = {'A': None, 'B': 'A', 'D': 'A', 'C': 'B', 'E': 'D', 'F': 'C'}
output: list[str] = get_path("A", "F", parent_tree)
print(output)
assert output == ["A", "B", "C", "F"]
###
### AUTOGRADER TEST - DO NOT REMOVE
###


['A', 'B', 'C', 'F']


In [37]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###
###
### AUTOGRADER TEST - DO NOT REMOVE
###


**HW3.** *Undirected Graph:* Create a subclass of `SearchGraph` class called `SearchUGraph` for undirected graphs. All edges in undirected graphs are bidirectional (i.e. vertex1 <-> vertex2). 
This means that you need to override the `add_edge()` method.

In [38]:
class Vertex:
    def __init__(self, id_: str="") -> None:
        self.id_: str = id_  # Initialize the vertex ID
        self.neighbours: dict[Vertex, Number] = {}  # Initialize the neighbours dictionary with weights
    
    def add_neighbour(self, nbr_vertex: Vertex, weight: Number=0) -> None:
        self.neighbours[nbr_vertex] = weight  # Add a neighbour with the given weight
    
    def get_neighbours(self) -> list[Vertex]:
        return list(self.neighbours.keys())  # Return a list of neighbour vertices
    
    def get_weight(self, neighbour: Vertex) -> Optional[Number]:
        return self.neighbours.get(neighbour, None)  # Get the weight of the edge to the neighbour
    
    def __eq__(self, other) -> bool:
        return self.id_ == other.id_  # Check equality based on vertex ID
    
    def __lt__(self, other) -> bool:
        return self.id_ < other.id_  # Compare vertices based on their IDs
    
    def __hash__(self) -> int:
        return hash(self.id_)  # Return the hash of the vertex ID
    
    def __str__(self) -> str:
        neighbours_ids = ', '.join([nbr.id_ for nbr in self.neighbours])  # Get a string of neighbour IDs
        return f"Vertex {self.id_} is connected to: {neighbours_ids}"  # Return a string representation of the vertex

In [39]:
import sys

class SearchVertex(Vertex):
    def __init__(self, id_: str="") -> None:
        super().__init__(id_)  # Call the constructor of the parent class Vertex
        self.colour: str = "white"  # Initialize the colour of the vertex to white (unvisited)
        self.d: int = sys.maxsize  # Initialize the discovery time to the maximum possible value
        self.f: int = sys.maxsize  # Initialize the finishing time to the maximum possible value
        self.parent: Optional[Vertex] = None  # Initialize the parent of the vertex to None


In [40]:
class Graph:
    def __init__(self) -> None:
        self.vertices: dict[str, Vertex] = {}  # Initialize an empty dictionary to store vertices
        
    @property
    def num_vertices(self) -> int:
        return len(self.vertices)  # Return the number of vertices in the graph

    def _create_vertex(self, id_: str) -> Vertex:
        return Vertex(id_)  # Create a new vertex with the given ID

    def add_vertex(self, id_: str) -> None:
        if id_ not in self.vertices:
            self.vertices[id_] = self._create_vertex(id_)  # Add the vertex to the graph if it doesn't already exist

    def get_vertex(self, id_: str) -> Optional[Vertex]:
        return self.vertices.get(id_, None)  # Retrieve the vertex with the given ID, or None if it doesn't exist

    def add_edge(self, start_v: str, end_v: str, weight: Number=0) -> None:
        if start_v not in self.vertices:
            self.add_vertex(start_v)  # Add the start vertex if it doesn't exist
        if end_v not in self.vertices:
            self.add_vertex(end_v)  # Add the end vertex if it doesn't exist
        self.vertices[start_v].add_neighbour(self.vertices[end_v], weight)  # Add the end vertex as a neighbour to the start vertex

    def get_neighbours(self, id_: str) -> list[str]:
        vertex = self.get_vertex(id_)  # Retrieve the vertex with the given ID
        if vertex:
            return [nbr.id_ for nbr in vertex.get_neighbours()]  # Return a list of neighbour IDs
        return []  # Return an empty list if the vertex doesn't exist
    
    def __contains__(self, val: str) -> bool:
        return val in self.vertices.keys()  # Check if a vertex with the given ID exists in the graph
    
    def __iter__(self):
        for k, v in self.vertices.items():
            yield v  # Allow iteration over the vertices in the graph


In [41]:
class SearchGraph(Graph):
    def _create_vertex(self, id_: str) -> SearchVertex:
        return SearchVertex(id_)


In [42]:
class SearchUGraph(SearchGraph):
    def add_edge(self, start_v: str, end_v: str, weight: Number=0) -> None:
        # If the start vertex is not in the graph, add it
        if start_v not in self.vertices:
            self.add_vertex(start_v)
        # If the end vertex is not in the graph, add it
        if end_v not in self.vertices:
            self.add_vertex(end_v)
        # Add the end vertex as a neighbour to the start vertex with the given weight
        self.vertices[start_v].add_neighbour(self.vertices[end_v], weight)
        # Add the start vertex as a neighbour to the end vertex with the given weight (since the graph is undirected)
        self.vertices[end_v].add_neighbour(self.vertices[start_v], weight)

In [43]:
g2: SearchUGraph = SearchUGraph()
assert g2.vertices == {} and g2.num_vertices == 0
g2.add_vertex("L")
g2.add_vertex("I")
g2.add_vertex("S")
g2.add_vertex("P")
assert g2.num_vertices == 4
assert "L" in g2
assert "I" in g2
assert "S" in g2
assert "P" in g2
g2.add_edge("L", "I")
g2.add_edge("I", "S")
g2.add_edge("S", "P")
assert sorted(g2.get_neighbours("L")) == ["I"]
assert sorted(g2.get_neighbours("I")) == ["L", "S"]
assert sorted(g2.get_neighbours("S")) == ["I", "P"]
assert sorted(g2.get_neighbours("P")) == ["S"]
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [44]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###
###
### AUTOGRADER TEST - DO NOT REMOVE
###


**HW4.** This homework assumes you have completed `TraverseBFS` class. Paste the code below and you will see that the class `TraverseBFS` works with undirected graph represented by `SearchUGraph` class.

In [45]:
class Queue:
    def __init__(self) -> None:
        self.__items: list[Any] = []  # Initialize an empty list to store queue items
    
    def enqueue(self, item: Any) -> None:
        self.__items.append(item)  # Add an item to the end of the queue

    def dequeue(self) -> Any:
        if self.is_empty:
            return None  # Return None if the queue is empty
        return self.__items.pop(0)  # Remove and return the item from the front of the queue
    
    def peek(self) -> Any:
        if self.is_empty:
            return None  # Return None if the queue is empty
        return self.__items[0]  # Return the item at the front of the queue without removing it
    
    @property
    def is_empty(self) -> bool:
        return len(self.__items) == 0  # Return True if the queue is empty, otherwise False
    
    @property
    def size(self) -> int:
        return len(self.__items)  # Return the number of items in the queue


In [46]:
import sys

class TraverseGraph:
    def __init__(self, g: SearchGraph) -> None:
        self.graph = g  # Initialize the graph attribute with the given SearchGraph object
    
    def clear_vertices(self) -> None:
        for vertex in self.graph.vertices.values():
            if isinstance(vertex, SearchVertex):
                vertex.colour = "white"  # Reset the colour to white (unvisited)
                vertex.d = sys.maxsize  # Reset the discovery time to the maximum possible value
                vertex.f = sys.maxsize  # Reset the finishing time to the maximum possible value
                vertex.parent = None  # Reset the parent to None
    
    def __iter__(self) -> Iterator:
        return iter([v for v in self.graph])  # Return an iterator over the vertices in the graph
    
    def __len__(self) -> int:
        return len([v for v in self.graph.vertices])  # Return the number of vertices in the graph


In [47]:
class TraverseBFS(TraverseGraph):

    def search_from(self, start: str) -> None:
        start_vertex = self.graph.get_vertex(start)
        if not isinstance(start_vertex, SearchVertex):
            return
        
        self.clear_vertices()  # Clear the vertices' properties
        start_vertex.d = 0  # Set the discovery time of the start vertex to 0
        start_vertex.colour = "grey"  # Mark the start vertex as being visited
        queue = Queue()  # Initialize a queue for BFS
        queue.enqueue(start_vertex)  # Enqueue the start vertex
        
        while not queue.is_empty:
            current_vertex = queue.dequeue()  # Dequeue the current vertex
            if not isinstance(current_vertex, SearchVertex):
                continue
            for neighbour in current_vertex.get_neighbours():
                if not isinstance(neighbour, SearchVertex):
                    continue
                if neighbour.colour == "white":  # If the neighbour has not been visited
                    neighbour.colour = "grey"  # Mark the neighbour as being visited
                    neighbour.d = current_vertex.d + 1  # Set the discovery time of the neighbour
                    neighbour.parent = current_vertex  # Set the parent of the neighbour
                    queue.enqueue(neighbour)  # Enqueue the neighbour
            current_vertex.colour = "black"  # Mark the current vertex as fully visited

    def get_shortest_path(self, start: str, dest: str) -> Optional[list[str]]:
        result: list[str] = []
        self.get_path(start, dest, result)  # Get the path from start to dest
        if not result:
            return None
        return result

    def get_path(self, start: str, dest: str, result: list[str]) -> None:
        start_vertex = self.graph.get_vertex(start)
        dest_vertex = self.graph.get_vertex(dest)
        
        if not isinstance(start_vertex, SearchVertex) or not isinstance(dest_vertex, SearchVertex):
            return None
        
        if start_vertex.d != 0:
            self.search_from(start)  # Perform BFS from the start vertex
        
        if start == dest:
            result.append(start)  # Add the start vertex to the result
        elif dest_vertex.parent is None:
            result.append("No Path")  # Add "No Path" if there is no path to the destination
        else:
            self.get_path(start, dest_vertex.parent.id_, result)  # Recursively get the path
            result.append(dest)  # Add the destination vertex to the result


In [48]:
g2: SearchUGraph = SearchUGraph()
g2.add_vertex("r")
g2.add_vertex("s")
g2.add_vertex("t")
g2.add_vertex("u")
g2.add_vertex("v")
g2.add_vertex("w")
g2.add_vertex("x")
g2.add_vertex("y")
g2.add_vertex("z")
g2.add_edge("r", "s")
g2.add_edge("r", "v")
g2.add_edge("s", "w")
g2.add_edge("t", "u")
g2.add_edge("t", "x")
g2.add_edge("t", "w")
g2.add_edge("u", "t")
g2.add_edge("u", "x")
g2.add_edge("u", "y")
g2.add_edge("v", "r")
g2.add_edge("w", "s")
g2.add_edge("w", "t")
g2.add_edge("w", "x")
g2.add_edge("x", "w")
g2.add_edge("x", "t")
g2.add_edge("x", "u")
g2.add_edge("x", "y")
g2.add_edge("y", "u")
g2.add_edge("y", "x")
gs: TraverseBFS = TraverseBFS(g2)
gs.search_from("s")
assert cast(SearchVertex, gs.graph.get_vertex("s")).d == 0
assert cast(SearchVertex, gs.graph.get_vertex("t")).d == 2
assert cast(SearchVertex, gs.graph.get_vertex("u")).d == 3
assert cast(SearchVertex, gs.graph.get_vertex("v")).d == 2
assert cast(SearchVertex, gs.graph.get_vertex("w")).d == 1
assert cast(SearchVertex, gs.graph.get_vertex("x")).d == 2
assert cast(SearchVertex, gs.graph.get_vertex("y")).d == 3
assert cast(SearchVertex, gs.graph.get_vertex("r")).d == 1
ans: Optional[list[str]] = gs.get_shortest_path("s", "u")
assert ans == ["s", "w", "t", "u"] or ans == ["s", "w", "x", "u"]
print(ans)
ans: Optional[list[str]] = gs.get_shortest_path("v", "s")
assert ans == ["v", "r", "s"]
print(ans)

ans: Optional[list[str]] = gs.get_shortest_path("v", "y")
print(ans)

assert ans == ["v", "r", "s", "w", "x", "y"]
###
### AUTOGRADER TEST - DO NOT REMOVE
###


['s', 'w', 't', 'u']
['v', 'r', 's']
['v', 'r', 's', 'w', 'x', 'y']


In [49]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###
###
### AUTOGRADER TEST - DO NOT REMOVE
###


**HW5.** *Depth-First-Search:* Create a class `TraverseDFS` as a child class of `TraverseGraph` to implement Depth-First-Search algorithm. You should add one additional attribute:
- `time`: which is an attribute to record the discovery time and the finishing time.

The class should also implement the following methods:
- `search()`: which modifies the vertices' properties such as `colour`, `d`, and `parent` following Depth-First-Search algorithm.
- `dfs_visit(vert)`: which is the recursive method for visiting a vertex in Depth-First-Search algorithm.

In [50]:
import sys

class TraverseDFS(TraverseGraph):
    def __init__(self, g: SearchGraph) -> None:
        super().__init__(g)
        self.time: int = 0  # Initialize the time attribute to 0
      
    def search(self) -> None:
        self.clear_vertices()  # Clear the vertices' properties
        self.time = 0  # Reset the time to 0
        for vertex in self.graph:
            # If the vertex is a SearchVertex and has not been visited (colour is white)
            if isinstance(vertex, SearchVertex) and vertex.colour == "white":
                self.dfs_visit(vertex)  # Perform DFS visit on the vertex
    
    def dfs_visit(self, vert: SearchVertex) -> None:
        self.time += 1  # Increment the time
        vert.d = self.time  # Set the discovery time of the vertex
        vert.colour = "grey"  # Mark the vertex as being visited
        for neighbour in vert.get_neighbours():
            # If the neighbour is a SearchVertex and has not been visited (colour is white)
            if isinstance(neighbour, SearchVertex) and neighbour.colour == "white":
                neighbour.parent = vert  # Set the parent of the neighbour to the current vertex
                self.dfs_visit(neighbour)  # Recursively visit the neighbour
        vert.colour = "black"  # Mark the vertex as fully visited
        self.time += 1  # Increment the time
        vert.f = self.time  # Set the finishing time of the vertex

In [51]:
g4: SearchGraph = SearchGraph()
g4.add_vertex("e")
g4.add_vertex("m")
g4.add_vertex("a")
g4.add_vertex("c")
g4.add_vertex("s")
g4.add_edge("e", "m")
g4.add_edge("m", "a")
g4.add_edge("a", "c")
g4.add_edge("c", "s")
gs: TraverseDFS = TraverseDFS(g4)
gs.search()
assert cast(SearchVertex, gs.graph.get_vertex("e")).parent == None 
assert cast(SearchVertex, gs.graph.get_vertex("m")).parent == gs.graph.get_vertex("e")

assert cast(SearchVertex, gs.graph.get_vertex("m")).d == 2 and cast(SearchVertex, gs.graph.get_vertex("a")).f == 8
assert cast(SearchVertex, gs.graph.get_vertex("c")).d == 4 and cast(SearchVertex, gs.graph.get_vertex("s")).f == 6
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [52]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###
###
### AUTOGRADER TEST - DO NOT REMOVE
###


**HW6.** *Topological Sort:* Create a function that takes in a `TraverseDFS` object to perform a topological sort:
- `topological_sort(g)`: which takes in a `TraverseDFS` object and returns a list of `SearchVertex` objects sorted based on their `f` attribute. This method should call the `search()` method of the `TraverseDFS` to first calculate the `f` attribute of the vertices. Note that you need to copy the `TraverseDFS` object of your input argument so as not to mutate the object.

In [53]:
import sys 
import copy 

def topological_sort(g: TraverseDFS) -> Iterable[SearchVertex]:
    g_copy = copy.deepcopy(g)  # Create a deep copy of the TraverseDFS object to avoid mutating the original graph
    g_copy.search()  # Perform a depth-first search to calculate discovery and finishing times
    
    # Sort the vertices based on their finishing times in descending order
    sorted_vertices: list[SearchVertex] = sorted(
        (v for v in g_copy.graph.vertices.values() if isinstance(v, SearchVertex)), 
        key=lambda v: v.f, 
        reverse=True
    )
    
    return sorted_vertices  # Return the sorted list of vertices


In [54]:
import copy
g: SearchGraph = SearchGraph()
g.add_vertex("3/4 cup milk")
g.add_vertex("1 egg")
g.add_vertex("1 tbl oil")
g.add_vertex("1 cup mix")
g.add_vertex("heat syrup")
g.add_vertex("heat griddle")
g.add_vertex("pour 1/4 cup")
g.add_vertex("turn when bubbly")
g.add_vertex("eat")
g.add_edge("3/4 cup milk", "1 cup mix")
g.add_edge("1 egg", "1 cup mix")
g.add_edge("1 tbl oil", "1 cup mix")
g.add_edge("1 cup mix", "heat syrup")
g.add_edge("1 cup mix", "pour 1/4 cup")
g.add_edge("heat griddle", "pour 1/4 cup")
g.add_edge("pour 1/4 cup", "turn when bubbly")
g.add_edge("turn when bubbly", "eat")
g.add_edge("heat syrup", "eat")
gs: TraverseDFS = TraverseDFS(g)  

path: Iterable[SearchVertex] = topological_sort(gs)
ans: list[int] = [v.f for v in copy.copy(path)]
assert ans == [18, 16, 14, 12, 11, 10, 9, 6, 5]
ans: list[str] = [v.id_ for v in copy.copy(path)]
assert ans == ['heat griddle', '1 tbl oil', '1 egg', '3/4 cup milk', '1 cup mix', 'pour 1/4 cup', 'turn when bubbly', 'heat syrup', 'eat']
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [55]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###
###
### AUTOGRADER TEST - DO NOT REMOVE
###
